In [12]:
!pip install tensorflow-transform

     |████████████████████████████████| 327kB 2.6MB/s 
     |████████████████████████████████| 2.0MB 4.2MB/s 
     |████████████████████████████████| 112kB 17.6MB/s 
     |████████████████████████████████| 3.4MB 16.9MB/s 
     |████████████████████████████████| 63.2MB 60kB/s 
     |████████████████████████████████| 1.4MB 47.5MB/s 
     |████████████████████████████████| 829kB 26.6MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 153kB 52.4MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 235kB 35.2MB/s 
     |████████████████████████████████| 184kB 45.8MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 122kB 43.6MB/s 
     |████████████████████████████████| 153kB 52.4MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
  

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import scipy.io
import numpy as np
import tensorflow as tf
import tensorflow_transform as tft

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
a = scipy.io.loadmat("/content/gdrive/My Drive/Sub/nonflipped/keep_30/Subpre/1.mat") 
# new = a['freqcohpersub']
new = a['M']
# new = nor['freqcohpersub'] 
Patient = []
P = []
for i in range (30):
    tmp = []
    for j in range (50):
        tmp.append(new[0,i][0,j])
    Patient.append(tmp)
# all range of herz of all patients' data   24,30,116,116       
Patient = np.array(Patient) 
print(Patient.shape)

(30, 50, 116, 116)


In [ ]:
#这个把 24个病人的 pre post fu 串到一起 例子：病人1 的50pre + 50post +50fu = 150个   (30, 150, 116, 116)

for Num in range(1,25):
  pre = scipy.io.loadmat('/content/gdrive/My Drive/Sub/nonflipped/keep_30/Subpre/' + str(Num) + '.mat') 
  post = scipy.io.loadmat('/content/gdrive/My Drive/Sub/nonflipped/keep_30/Subpost/' + str(Num) + '.mat') 
  fu = scipy.io.loadmat('/content/gdrive/My Drive/Sub/nonflipped/keep_30/Subfu/' + str(Num) + '.mat') 
  # if 'y' in nor:
  #   print(Num)
  # tmp1 = pre['freqcohpersub'] 
  # tmp2 = post['freqcohpersub'] 
  # tmp3 = fu['freqcohpersub'] 
  tmp1 = pre['M'] 
  tmp2 = post['M'] 
  tmp3 = fu['M'] 

  Patient = []
  P = []
  for i in range (30):
      tmp = []
      for j in range (50):
          tmp.append(tmp1[0,i][0,j])
      Patient.append(tmp)
  # all range of herz of all patients' data   24,30,116,116       
  Patient1 = np.array(Patient) 
  # print(Num)
  # print(Patient1.shape)  

  Patient = []
  P = []
  for i in range (30):
      tmp = []
      for j in range (50):
          tmp.append(tmp2[0,i][0,j])
      Patient.append(tmp)
  # all range of herz of all patients' data   24,30,116,116       
  Patient2 = np.array(Patient) 
  # print(Num)
  # print(Patient2.shape) 

  Patient = []
  P = []
  for i in range (30):
      tmp = []
      for j in range (50):
          tmp.append(tmp3[0,i][0,j])
      Patient.append(tmp)
  # all range of herz of all patients' data   24,30,116,116       
  Patient3 = np.array(Patient) 
  # print(Num)
  # print(Patient3.shape) 

  print(Num)
  PATIENT = np.concatenate((Patient1,Patient2,Patient3), axis = 1)   #形状为(30, 150, 116, 116)
  np.save('/content/gdrive/My Drive/Newdata/ordered_by_patient/connectivity/nonflipped_keep30_150_'+str(Num), PATIENT)



# Numpy simple GCN

In [2]:
a = np.load('/content/gdrive/My Drive/Newdata/ordered_by_patient/connectivity/nonflipped_keep30_150_1.npy')
a = np.reshape(a,(150,30,116,116))

In [3]:
b = np.load('/content/gdrive/My Drive/strength/nonflipped/keep_30/three/1.npy')
b = np.reshape(b,(150,30,116))
print(b.shape)

(150, 30, 116)


In [4]:
X = b[0,0]  #为初始输入值
X = np.reshape(X,(116,1))


In [5]:
A = a[0,0] #就是邻接矩阵
c = tf.where(b>0,1,0) #所有大于0的都替换成1用来表示两个node之间有连接
I = np.matrix(np.eye(A.shape[0])) #单位矩阵 
A_hat = A + I 
# 生成对应的度矩阵
D_hat = np.array(np.sum(c, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat)) 

In [10]:
 # 初始化随机生成两层权重数值
tf.compat.v1.disable_eager_execution()
from scipy.stats import zscore
W_1 = np.random.normal(  # Drawn samples from the parameterized normal distribution.返回形状为（116，4）的参数化的正态分布值
    loc=0, scale=1, size=(A.shape[0], 1))
print(W_1.shape)
W_2 = np.random.normal(
    loc=0, scale=1,size=(W_1.shape[1], 1)) #生成（4,2）形状的正态分布矩阵
print(W_2.shape)
# print((D_hat**-1 * A_hat * X).shape)
def gcn_layer(A_hat, D_hat, X, W):
  tmp = D_hat**-1 * A_hat * X * W 
  print(tmp)
  tmp = scipy.stats.zscore(tmp)
  # tmp = tf.linalg.normalize(tmp)[1]
  # tmp = tft.scale_to_z_score(tmp,elementwise=False,output_dtype=tf.float32)
  print(tmp)
  return tf.nn.relu(tmp)
    # return np.array(tf.nn.relu(D_hat**-1 * A_hat * X * W))

H_1 = gcn_layer(A_hat, D_hat, A, W_1)
# # H_1 = np.array(H_1)
# print(H_1)
# print((D_hat**-1*A_hat).shape)
# print(((D_hat**-1*A_hat)*H_1).shape)
# H_2 = gcn_layer(A_hat, D_hat, H_1, W_2)

# print(H_2)

(116, 1)
(1, 1)
[[-0.0119924 ]
 [-0.01853103]
 [-0.01561449]
 [-0.02310057]
 [-0.01411506]
 [-0.01235617]
 [-0.0127825 ]
 [-0.0224795 ]
 [-0.01246172]
 [-0.01077896]
 [-0.00235496]
 [-0.01510738]
 [-0.00298563]
 [-0.01977296]
 [-0.00631188]
 [-0.02061514]
 [-0.00221867]
 [-0.0239597 ]
 [-0.0196213 ]
 [-0.02191893]
 [-0.01118971]
 [-0.0160788 ]
 [-0.01649891]
 [-0.0201256 ]
 [-0.01991563]
 [-0.01549939]
 [-0.01618333]
 [-0.01330171]
 [-0.00361567]
 [-0.02026212]
 [-0.01579786]
 [-0.01577446]
 [-0.01373565]
 [-0.01623975]
 [-0.0077627 ]
 [-0.01073176]
 [-0.00048962]
 [-0.02088631]
 [ 0.00064936]
 [-0.02398407]
 [-0.00185596]
 [-0.02225624]
 [-0.00392776]
 [-0.01296318]
 [-0.00854179]
 [-0.00936562]
 [-0.00335387]
 [-0.01499081]
 [-0.00890653]
 [-0.00775221]
 [-0.00424263]
 [-0.00960076]
 [-0.00130789]
 [-0.02069019]
 [-0.00275129]
 [-0.02173369]
 [-0.01079708]
 [-0.01168476]
 [-0.01347948]
 [-0.01162474]
 [-0.01488983]
 [-0.00570534]
 [-0.01229269]
 [-0.01328538]
 [-0.01243578]
 [-0.0069

TypeError: ignored

In [ ]:
# tf.compat.v1.disable_eager_execution()
# from sklearn.preprocessing import normalize
# H_3 = normalize(H_2, norm='l1', copy=True, return_norm=False)
# print(H_3)
H_5 = tft.scale_to_z_score(H_1)
tf.print(H_5)
# for i in H_5: 
#   print(i)
# from scipy.stats import zscore
# H_4 = zscore(H_2)
# print(H_4)

<tf.Operation 'PrintV2_1' type=PrintV2>